In [2]:
from multiprocessing import Lock, Process, Queue, current_process
import time
import queue
import string
import requests
from lxml import html
from robot import Robot
import pickle
MAX_NUM = 2

In [3]:
def get_parsed_page(url):
    return html.fromstring(requests.get(url).content)

def get_links(parsed_page, url, url_doc, robot):
    result = []
    post_urls = parsed_page.xpath('//a/@href')
    for post_url in post_urls:
        if len(post_url) == 0:
            continue
        if post_url[0] == '/':
            post_url = url + (post_url[1:] if post_url[-1] == '/' else post_url)
        if post_url not in url_doc and robot.is_allowed(post_url):
            result.append(post_url)
    return result

In [9]:
def create_env(url):
    robot = Robot()
    url_doc = {}
    to_do = Queue()
    to_do.put(url)

    return to_do, url_doc, robot

In [10]:
def do_job(to_do, todo_neib, url_doc, letter, robot, num_processes):
    while True:
        try:
            url = to_do.get_nowait()
        except queue.Empty:
            break
        else:
            if url[8] in letter:
                print(len(url_doc))
                if len(url_doc) >= MAX_NUM:
                    break

                if url in url_doc or not robot.is_allowed(url):
                    continue

                parsed_page = get_parsed_page(url)
                url_doc[url] = parsed_page
                for url_new in get_links(parsed_page, url, url_doc, robot):
                    to_do.put(url_new)
                if len(url_doc) % 10 == 0:
                    print(len(url_doc), num_processes)
                    with open('filename'+str(num_processes)+'_'+str(len(url_doc)) +'.pickle', 'wb') as handle:
                        pickle.dump(url_doc, handle, protocol=pickle.HIGHEST_PROTOCOL)

            else:
                todo_neib.put(url)
    return True          



In [11]:
number_of_processes = 2
alphabet = string.ascii_lowercase
processes = []


to_do1, url_doc1, robot1 = create_env('https://search.crossref.org/')
to_do0, url_doc0, robot0 = create_env('http://plos.org/')
z = int(len(alphabet)/2)

letter0 = alphabet[:z]
p = Process(target=do_job, args=(to_do0, to_do1, url_doc0, letter0, robot0, 0))
processes.append(p)
p.start()

letter1 = alphabet[z:]
p = Process(target=do_job, args=(to_do1, to_do0, url_doc1, letter1, robot1, 1))


processes.append(p)
p.start()

# completing process
for p in processes:
    p.join()


0
0


In [7]:
url_doc0

{'http://plos.org/': <Element html at 0x107703ea8>}

In [8]:
url_doc1

{'https://search.crossref.org/': <Element html at 0x1076f06d8>}